Data Cleaning Phase

In [29]:
import pandas as pd

In [ ]:
dt = pd.read_csv('vscode_bugs.csv')
print(dt.head())
print(f"\n {dt.info()}")
print(f"\n{dt.describe()}")

   Issue id                                            Summary  \
0    223706  Configure unassigned keybindings, command name...   
1    223658  Some suggestions are missing on Windows with pwsh   
2    223641  Cannot split a terminal without a group [objec...   
3    223622  SCM - history graph handling of first commit i...   
4    223607          SCM - history graph using incorrect color   

                     Created                   Resolved  \
0  2024-07-25 14:34:59+00:00                        NaN   
1  2024-07-25 13:06:40+00:00  2024-07-25 16:14:57+00:00   
2  2024-07-25 12:19:09+00:00                        NaN   
3  2024-07-25 10:44:47+00:00                        NaN   
4  2024-07-25 09:43:02+00:00  2024-07-25 11:02:46+00:00   

                                         Description Resolution  
0  - Place cursor in the panel chat\r\n- Open the...        NaN  
1  I was trying to verify https://github.com/micr...  completed  
2  ![image](https://github.com/user-attachments/a.

In [31]:
print(dt.duplicated().sum()) #to count the number of duplicated rows if exist
dt.drop_duplicates(inplace=True)   #to remove the duplicates

print(dt.duplicated().sum())   #verify if duplicates still exist

1
0


In [32]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

dt['Created'] = pd.to_datetime(dt['Created'], errors='coerce', dayfirst=True)     #this converts the day and time in the Created column to readable datetime object
dt = dt.dropna(subset=['Created'])    #if a row fails to covert, drop the row

#for rows that have missing decriptions
dt['Description'] = dt['Description'].fillna('')

dt['Notes'] = dt['Summary'] + '  ' + dt['Description']    #combine the summary and description columns into one text column
print(dt['Notes'].head())        #check out the data for validity



C:\Users\Rest\AppData\Local\Temp\ipykernel_14404\1698058673.py:5: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt['Created'] = pd.to_datetime(dt['Created'], errors='coerce', dayfirst=True)     #this converts the day and time in the Created column to readable datetime object


0    Configure unassigned keybindings, command name...
1    Some suggestions are missing on Windows with p...
2    Cannot split a terminal without a group [objec...
3    SCM - history graph handling of first commit i...
4    SCM - history graph using incorrect color  ![I...
Name: Notes, dtype: object


In [33]:
import re

#remove special characters
dt['Notes'] = dt['Notes'].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]', '', x))        #this takes care of special characters using the re.sub() function
print(dt['Notes'].head())       #verify if those changes have been made


0    Configure unassigned keybindings command name ...
1    Some suggestions are missing on Windows with p...
2    Cannot split a terminal without a group object...
3    SCM  history graph handling of first commit in...
4    SCM  history graph using incorrect color  Imag...
Name: Notes, dtype: object


In [34]:
#normalize to lowercase
dt['Notes'] = dt['Notes'].apply(lambda x: x.lower())
print(dt['Notes'].head())       #verify if those changes have been made


0    configure unassigned keybindings command name ...
1    some suggestions are missing on windows with p...
2    cannot split a terminal without a group object...
3    scm  history graph handling of first commit in...
4    scm  history graph using incorrect color  imag...
Name: Notes, dtype: object
